<a href="https://colab.research.google.com/github/gafm-ai/astro_learn/blob/main/prepareData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=5e87adedbc136e2e921803ea1f21908f898f2f87a3fbdcee0c2b2457a78920bf
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import logging
import pytz
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Fonction pour vérifier le timeframe
def verify_timeframe(timeframe):
    valid_timeframes = ['1min', '5min', '15min', '30min', '1H', '4H', '1D']
    if timeframe not in valid_timeframes:
        logger.error(f"Timeframe invalide : {timeframe}. Veuillez choisir parmi {valid_timeframes}.")
        raise ValueError("Timeframe invalide.")
    logger.info(f"Timeframe sélectionné : {timeframe}")

# Fonction pour charger et préparer les données
def load_and_prepare_data(file_path, timeframe):
    # Charger les données
    df = pd.read_csv(file_path, parse_dates=['timestamp'])

    # Renommer les colonnes
    #df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'}, inplace=True)

    # Utiliser 'timestamp' comme index
    df.set_index('timestamp', inplace=True)

    # Ordonner les données par date croissante
    df = df.sort_index()

    # S'assurer que l'index est en timezone UTC
    if df.index.tz is None:
        # Si ce n'est pas le cas, localiser en UTC
        df.index = df.index.tz_localize(pytz.utc)
    else:
        # Si l'index a déjà un fuseau horaire, convertir en UTC
        df.index = df.index.tz_convert(pytz.utc)

    print(df.head())

    print(f"Données chargées : {df.shape[0]} lignes.")
    return df

# Fonction pour gérer les gaps temporels et combler les données manquantes
def handle_time_gaps(df, timeframe):
    # Définir les jours ouvrables (lundi=0, ..., vendredi=4)
    business_days = [0, 1, 2, 3, 4]

    # Créer un index complet sans week-ends
    full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq=timeframe, tz=pytz.utc)
    full_index = full_index[full_index.dayofweek.isin(business_days)]

    # Reindexer le DataFrame
    df = df.reindex(full_index)

    # Combler les données manquantes
    df['close'] = df['close'].ffill()
    df['open'] = df['open'].fillna(df['close'])
    df['high'] = df['high'].fillna(df['close'])
    df['low'] = df['low'].fillna(df['close'])
    df['volume'] = df['volume'].fillna(0)

    print("Données manquantes comblées.")
    return df

# Fonction pour calculer les indicateurs
def calculate_indicators(df):
    # Calculer les chandeliers Heikin-Ashi
    ha = pd.DataFrame(index=df.index)
    ha['HA_Close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    ha_open = ((df['open'].shift(1) + df['close'].shift(1)) / 2)
    ha['HA_Open'] = ha_open.fillna(df['open'])
    ha['HA_High'] = pd.concat([ha['HA_Open'], ha['HA_Close'], df['high']], axis=1).max(axis=1)
    ha['HA_Low'] = pd.concat([ha['HA_Open'], ha['HA_Close'], df['low']], axis=1).min(axis=1)


    print("Chandeliers Heikin-Ashi calculés.")

    # Calculer les SMA et leur lissage
    ha['SMA_60'] = ta.sma(ha['HA_Close'], length=60)
    ha['SMA_60_Smoothed'] = ta.sma(ha['SMA_60'], length=5)
    ha['SMA_20'] = ta.sma(ha['HA_Close'], length=20)
    ha['SMA_20_Smoothed'] = ta.sma(ha['SMA_20'], length=5)
    ha['SMA_5'] = ta.sma(ha['HA_Close'], length=5)
    ha['SMA_5_Smoothed'] = ta.sma(ha['SMA_5'], length=5)

    # Calculer les autres indicateurs

    ha[['TSI_13_25_13', 'TSIs_13_25_13']] = ta.tsi(ha['HA_Close']).values
    ha[['VTXP_14', 'VTXM_14']] = ta.vortex(ha['HA_High'], ha['HA_Low'], ha['HA_Close'], length=14).values
    ha['MFI_14'] = ta.mfi(ha['HA_High'], ha['HA_Low'], ha['HA_Close'], df['volume'])
    ha['AO_5_34'] = ta.ao(ha['HA_High'], ha['HA_Low'])

    ha['VWAP'] = ta.vwap(ha['HA_High'], ha['HA_Low'], ha['HA_Close'], df['volume'])

    # Ajouter des patterns avec talib
    #ha['Doji'] = talib.CDLDOJI(ha['HA_Open'], ha['HA_High'], ha['HA_Low'], ha['HA_Close'])

    print("Indicateurs calculés.")
    return ha

# Fonction pour détecter les croisements SMA
def detect_sma_crossings(ha):
    # Créer des colonnes booléennes pour les croisements
    ha['Cross_SMA_60'] = ((ha['SMA_60'] > ha['SMA_60_Smoothed']) & (ha['SMA_60'].shift(1) <= ha['SMA_60_Smoothed'].shift(1))) | \
                         ((ha['SMA_60'] < ha['SMA_60_Smoothed']) & (ha['SMA_60'].shift(1) >= ha['SMA_60_Smoothed'].shift(1)))

    ha['Cross_SMA_20'] = ((ha['SMA_20'] > ha['SMA_20_Smoothed']) & (ha['SMA_20'].shift(1) <= ha['SMA_20_Smoothed'].shift(1))) | \
                         ((ha['SMA_20'] < ha['SMA_20_Smoothed']) & (ha['SMA_20'].shift(1) >= ha['SMA_20_Smoothed'].shift(1)))

    ha['Cross_SMA_5'] = ((ha['SMA_5'] > ha['SMA_5_Smoothed']) & (ha['SMA_5'].shift(1) <= ha['SMA_5_Smoothed'].shift(1))) | \
                        ((ha['SMA_5'] < ha['SMA_5_Smoothed']) & (ha['SMA_5'].shift(1) >= ha['SMA_5_Smoothed'].shift(1)))

    ha['Cross_SMA_5_20'] = ((ha['SMA_5'] > ha['SMA_20']) & (ha['SMA_5'].shift(1) <= ha['SMA_20'].shift(1))) | \
                    ((ha['SMA_5'] < ha['SMA_20']) & (ha['SMA_5'].shift(1) >= ha['SMA_20'].shift(1)))

    ha['Cross_SMA_5_60'] = ((ha['SMA_5'] > ha['SMA_60']) & (ha['SMA_5'].shift(1) <= ha['SMA_60'].shift(1))) | \
                    ((ha['SMA_5'] < ha['SMA_60']) & (ha['SMA_5'].shift(1) >= ha['SMA_60'].shift(1)))

    ha['Cross_SMA_20_60'] = ((ha['SMA_20'] > ha['SMA_60']) & (ha['SMA_20'].shift(1) <= ha['SMA_60'].shift(1))) | \
                    ((ha['SMA_20'] < ha['SMA_60']) & (ha['SMA_20'].shift(1) >= ha['SMA_60'].shift(1)))

    return ha

# Fonction pour calculer les labels
def calculate_labels(df, pip_value=0.0001):
    # Fonction interne pour calculer les labels pour une direction
    def _calculate_directional_labels(df, pip_targets, periods, direction='buy'):
        labels = pd.DataFrame(index=df.index)
        for pip_target, period in zip(pip_targets, periods):
            if direction == 'buy':
                future_high = df['high'].shift(-1).rolling(window=period).max()
                target_price = df['close'] + (pip_target * pip_value)
                label = (future_high >= target_price).astype(int)
            elif direction == 'sell':
                future_low = df['low'].shift(-1).rolling(window=period).min()
                target_price = df['close'] - (pip_target * pip_value)
                label = (future_low <= target_price).astype(int)
            else:
                raise ValueError("Direction doit être 'buy' ou 'sell'")
            label_name = f"{direction}_{pip_target}pip_{period}bars"
            labels[label_name] = label
        return labels

    # Définir les pip targets et les périodes pour les achats
    pip_targets_buy = [1.5, 1.5, 2.5, 2.5, 5.5, 9]
    periods_buy = [2, 5, 5, 10, 10, 10]

    # Calculer les labels pour les achats
    labels_buy = _calculate_directional_labels(df, pip_targets_buy, periods_buy, direction='buy')
    print("Labels d'achat calculés.")

    # Définir les pip targets et les périodes pour les ventes
    pip_targets_sell = [1.5, 1.5, 2.5, 2.5, 5.5, 9]
    periods_sell = [2, 5, 5, 10, 10, 10]

    # Calculer les labels pour les ventes
    labels_sell = _calculate_directional_labels(df, pip_targets_sell, periods_sell, direction='sell')
    print("Labels de vente calculés.")

    # Combiner les labels
    labels = pd.concat([labels_buy, labels_sell], axis=1)
    return labels

In [ ]:
# Fonction principale pour exécuter le pipeline de traitement
def main(file_path, timeframe='1min', sequence_length=32):
    verify_timeframe(timeframe)
    df = load_and_prepare_data(file_path, timeframe)
    df = handle_time_gaps(df, timeframe)
    ha_df = calculate_indicators(df)

    # Joindre les données originales pour le calcul des labels
    data = ha_df.join(df[['open', 'high', 'low', 'close', 'volume']], how='left')

    # Supprimer les lignes avec des valeurs manquantes
    data.dropna(inplace=True)
    print(f"Données après suppression des valeurs manquantes : {data.shape[0]} lignes.")

    # Ajouter une colonne 'date' pour identifier le jour de trading
    data['date'] = data.index.date

    # Ajouter les colonnes 'hour', 'minute', 'second' pour le temps
    data['hour'] = data.index.hour
    data['minute'] = data.index.minute
    #data['second'] = data.index.second
    print(data.columns)
    # Calculer les labels
    labels = calculate_labels(data)
    data = data.join(labels)
    data.dropna(inplace=True)
    print(f"Données après ajout des labels : {data.shape[0]} lignes.")

    # Détecter les croisements SMA
    data = detect_sma_crossings(data)

    # Début de la création des séquences
    print("Début de la création des séquences.")

    # Préparer les features et labels
    feature_columns = ['HA_Close', 'HA_Open', 'HA_High', 'HA_Low', 'SMA_60', 'SMA_60_Smoothed',
       'SMA_20', 'SMA_20_Smoothed', 'SMA_5', 'SMA_5_Smoothed', 'TSI_13_25_13',
       'TSIs_13_25_13', 'VTXP_14', 'VTXM_14', 'MFI_14', 'AO_5_34', 'VWAP',
       'open', 'high', 'low', 'close', 'volume', 'hour', 'minute']
    label_columns = labels.columns.tolist()

    # Créer les séquences (fenêtres) de longueur 32
    X = []
    y = []
    json_data = []

    total_sequences = len(data) - sequence_length + 1
    print(f"Nombre total de séquences possibles : {total_sequences}")

    for i in range(total_sequences):
        sequence_data = data.iloc[i:i+sequence_length]
        last_row = sequence_data.iloc[-1]

        # Obtenir le timestamp de la dernière observation
        timestamp = last_row.name

        # S'assurer que timestamp est un objet Timestamp
        if not isinstance(timestamp, pd.Timestamp):
            timestamp = pd.to_datetime(timestamp)

        # Vérifier que la dernière observation est dans l'intervalle 7h-19h
        if timestamp.time() >= pd.to_datetime('07:00').time() and timestamp.time() <= pd.to_datetime('19:00').time():
            # Vérifier que toutes les observations de la séquence sont du même jour
            if sequence_data['date'].nunique() == 1:
                # Vérifier si un croisement SMA se produit à la dernière étape de la séquence
                if last_row['Cross_SMA_60'] or last_row['Cross_SMA_20'] or last_row['Cross_SMA_5'] or last_row['Cross_SMA_5_60'] or last_row['Cross_SMA_5_60'] or last_row['Cross_SMA_20_60']:
                    X_sequence = sequence_data[feature_columns]
                    y_sequence = sequence_data[label_columns].iloc[-1]
                    X.append(X_sequence.values)
                    y.append(y_sequence.values)

                    # Préparer les features avec le format souhaité
                    features_dict = {}
                    for col in feature_columns:
                        features_dict[col] = X_sequence[col].tolist()

                    # Préparer les labels avec les noms de colonnes
                    label_dict = y_sequence.to_dict()

                    # Préparer l'entrée pour le JSON
                    entry = {
                        'features': features_dict,
                        'labels': label_dict
                    }
                    json_data.append(entry)
        else:
            # Afficher une information si nécessaire
            if i % 100000 == 0:
                print(f"Séquence ignorée car hors de l'intervalle 7h-19h à l'index {timestamp}")

    X = np.array(X)
    y = np.array(y)
    print(f"Séquences finales créées : X.shape = {X.shape}, y.shape = {y.shape}")

    # Enregistrer les données au format .npy
    np.save('/content/drive/MyDrive/FLEURY/Finances/new32_1/X.npy', X)
    np.save('/content/drive/MyDrive/FLEURY/Finances/new32_1/y.npy', y)
    print("Données enregistrées au format .npy.")

    # Enregistrer les données au format JSON pour le LLM
    with open('/content/drive/MyDrive/FLEURY/Finances/new32_1/data.json', 'w') as json_file:
        json.dump(json_data, json_file)
    print("Données enregistrées au format JSON avec les features structurées.")


if __name__ == "__main__":
    main('/content/drive/MyDrive/FLEURY/Finances/data/OANDA_EUR_USD_1_20120101_20221231_price.csv', timeframe='5min', sequence_length=12)


                              open     high      low    close  volume
timestamp                                                            
2012-01-01 17:57:00+00:00  1.29525  1.29525  1.29425  1.29425       2
2012-01-01 17:58:00+00:00  1.29416  1.29416  1.29416  1.29416       1
2012-01-01 18:00:00+00:00  1.29409  1.29409  1.29409  1.29409       1
2012-01-01 18:01:00+00:00  1.29393  1.29403  1.29388  1.29403       4
2012-01-01 18:02:00+00:00  1.29408  1.29416  1.29263  1.29332      23
Données chargées : 3937808 lignes.
Données manquantes comblées.
Chandeliers Heikin-Ashi calculés.


<ipython-input-56-21be7b5c9fb8>:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 1.29459     0.          0.         ... 31.04024667 12.84369
 78.127885  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ha['MFI_14'] = ta.mfi(ha['HA_High'], ha['HA_Low'], ha['HA_Close'], df['volume'])
<ipython-input-56-21be7b5c9fb8>:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[  1.29460667   0.          25.88298333 ...  41.740205    66.35581
 120.91922833]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ha['MFI_14'] = ta.mfi(ha['HA_High'], ha['HA_Low'], ha['HA_Close'], df['volume'])
<ipython-input-56-21be7b5c9fb8>:90: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  ha['VWAP'] = ta.vwap(ha['HA_High'], ha['

Indicateurs calculés.
Données après suppression des valeurs manquantes : 809844 lignes.
Index(['HA_Close', 'HA_Open', 'HA_High', 'HA_Low', 'SMA_60', 'SMA_60_Smoothed',
       'SMA_20', 'SMA_20_Smoothed', 'SMA_5', 'SMA_5_Smoothed', 'TSI_13_25_13',
       'TSIs_13_25_13', 'VTXP_14', 'VTXM_14', 'MFI_14', 'AO_5_34', 'VWAP',
       'open', 'high', 'low', 'close', 'volume', 'date', 'hour', 'minute'],
      dtype='object')
Labels d'achat calculés.
Labels de vente calculés.
Données après ajout des labels : 809844 lignes.
Début de la création des séquences.
Nombre total de séquences possibles : 809833
Séquence ignorée car hors de l'intervalle 7h-19h à l'index 2012-01-02 06:12:00+00:00
Séquence ignorée car hors de l'intervalle 7h-19h à l'index 2014-09-19 02:27:00+00:00
Séquence ignorée car hors de l'intervalle 7h-19h à l'index 2016-02-01 06:42:00+00:00
Séquence ignorée car hors de l'intervalle 7h-19h à l'index 2017-06-09 00:42:00+00:00
Séquence ignorée car hors de l'intervalle 7h-19h à l'index 2